In [ ]:
import gate.data.medical.segmentation.medical_decathlon as medical_decathlon

In [ ]:
import os
import dotenv

# Load environment variables from .env file
dotenv.load_dotenv(
    dotenv_path="/disk/scratch_fast1/aantoni2/GATE/secrets/setup_variables.env"
)
import huggingface_hub

huggingface_hub.login()

In [ ]:
dataset = medical_decathlon.build_dataset(
    set_name="train",
    data_dir=os.environ.get("DATASET_DIR"),
    task_name="task01braintumour",
)

In [ ]:
for item in dataset:
    break

In [ ]:
print(list(item.keys()))

In [ ]:
import torch

image = (
    torch.stack([torch.tensor(i) for i in item["image"]])
    if isinstance(item["image"], list)
    else item["image"]
)
annotation = (
    torch.stack([torch.tensor(i) for i in item["label"]])
    if isinstance(item["label"], list)
    else item["label"]
)
image = image.permute(3, 0, 1, 2)[:, :3]
annotation = annotation.permute(0, 3, 1, 2)[0]

In [ ]:
print(image.shape)
print(annotation.shape)

In [ ]:
def show_batch(x: torch.Tensor, y: torch.Tensor) -> None:
    import torch
    from torchvision.utils import make_grid
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
    import numpy as np

    # Assuming x is a tensor of images with shape (B, C, H, W)
    # and y is a tensor of labels with shape (B, H, W), values from 0 to 255

    # Create a color map for 256 different classes
    cmap = plt.cm.get_cmap(
        "nipy_spectral", 256
    )  # You can choose any suitable colormap
    cmap_colors = cmap(np.arange(256))[
        :, :3
    ]  # Extract RGB values as a numpy array

    # Convert cmap_colors to a PyTorch tensor
    cmap_colors_tensor = torch.tensor(cmap_colors, dtype=torch.float32)

    # Overlay each image with its label
    overlaid_images = []
    for img, label in zip(x, y):
        # Normalize image for proper visualization
        img = (img - img.min()) / (img.max() - img.min())

        # Expand label dimensions and convert to RGB using the color map
        label_expanded = label.unsqueeze(0).repeat(3, 1, 1)  # Expand to CxHxW
        label_rgb = cmap_colors_tensor[label_expanded.long()]

        # Overlay label on image with transparency
        alpha = 0.4  # Transparency factor
        overlaid_img = (1 - alpha) * img + alpha * label_rgb
        overlaid_images.append(overlaid_img)

    # Create a grid of the overlaid images
    grid = make_grid(
        overlaid_images, nrow=20
    )  # Adjust the number of rows as needed

    # Permute grid for correct display
    grid = grid.permute((1, 2, 0))

    plt.figure(figsize=(12, 12))  # Adjust the size as needed
    plt.imshow(grid)
    plt.axis("off")  # Hide axes for better visualization
    plt.show()

In [ ]:
show_batch(image, annotation)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

# Assuming input_img is your batch of images and input_mask is your batch of masks
# For demonstration, we create a random image and mask
input_img = torch.rand(240, 240, 3)
input_mask = torch.randint(low=0, high=2, size=(240, 240))  # binary mask